# Resources:

* [extract clean text from html](https://stackoverflow.com/questions/328356/extracting-text-from-html-file-using-python)
* [extract clean text from pdf](https://www.geeksforgeeks.org/extract-text-from-pdf-file-using-python/)
* [word cloud tutorial](https://towardsdatascience.com/simple-wordcloud-in-python-2ae54a9f58e5)